In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import cross_val_score


clean_data= pd.read_csv('model_data.csv')
print(clean_data.columns)


Index(['Datum      ', 'Umsatz              ', 'Wettercode ', 'KielerWoche ',
       'Jahreszeit_1 ', 'Jahreszeit_2 ', 'Jahreszeit_3 ', 'Jahreszeit_4 ',
       'Wetter_Kalt ', 'Wetter_Mild ', 'Wetter_Warm ', 'Feiertag ',
       'KielLauf ', 'Kieler_Triathlon ', 'Fußball ', 'PaycheckEffect ',
       'Sensitivity_High ', 'Sensitivity_Moderate ', 'Wind_Nicht windig ',
       'Wind_Windig ', 'Wind_Zu windig ', 'Inflation_Kategorisierung_Neutral ',
       'Inflation_Kategorisierung_Positiv ',
       'Inflation_Kategorisierung_vormonat_Neutral ',
       'Inflation_Kategorisierung_vormonat_Positiv ', 'gefühl_Kalt ',
       'gefühl_Mild ', 'gefühl_Warm ', 'Wochentag_0 ', 'Wochentag_1 ',
       'Wochentag_2 ', 'Wochentag_3 ', 'Wochentag_4 ', 'Wochentag_5 ',
       'Wochentag_6 ', 'Warengruppe_1 ', 'Warengruppe_2 ', 'Warengruppe_3 ',
       'Warengruppe_4 ', 'Warengruppe_5 ', 'Warengruppe_6'],
      dtype='object')


In [12]:

# Features und Ziel definieren
# Beispiel: Ziel ist "Umsatz", Features sind andere numerische Spalten
X = clean_data.drop(columns=['Datum      ','Umsatz              ','Wettercode '])  # Features: Alle außer 'Umsatz' und 'Datum'
y = clean_data['Umsatz              ']  # Ziel: Umsatz

# Prüfen, ob Kategorien in numerische Werte konvertiert werden müssen
X = pd.get_dummies(X, drop_first=True)

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Lineares Regressionsmodell trainieren
model = LinearRegression()
model.fit(X_train, y_train)

# Vorhersagen
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Modellbewertung
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

# Ergebnisse ausgeben
print(f"Trainingsdaten: MSE = {mse_train:.2f}, R² = {r2_train:.2f}")
print(f"Testdaten: MSE = {mse_test:.2f}, R² = {r2_test:.2f}")

# Wichtige Koeffizienten und Features
coefficients = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print("\nKoeffizienten der Features:")
print(coefficients.sort_values(by='Coefficient', ascending=False))

scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(scores)

Trainingsdaten: MSE = 5370.82, R² = 0.72
Testdaten: MSE = 6288.06, R² = 0.70

Koeffizienten der Features:
                                        Feature   Coefficient
14                        Sensitivity_Moderate   1.522505e+15
31                                 Wochentag_6   1.442371e+15
30                                 Wochentag_5   1.442371e+15
29                                 Wochentag_4   1.442371e+15
25                                 Wochentag_0   1.442371e+15
26                                 Wochentag_1   1.442371e+15
27                                 Wochentag_2   1.442371e+15
28                                 Wochentag_3   1.442371e+15
33                               Warengruppe_2   4.094020e+14
32                               Warengruppe_1   4.094020e+14
37                                Warengruppe_6  4.094020e+14
8                                     Feiertag   6.389855e+01
18           Inflation_Kategorisierung_Neutral   2.874857e+01
9                         

In [ ]:
import pandas as pd

# Beispiel-Datensatz laden
data = pd.read_csv('model_data.csv')  # Datensatz laden

# Konvertiert die Datumswerte (falls sie nicht schon datetime-Format haben)
data['Datum      '] = pd.to_datetime(data['Datum      '], format='ISO8601')
# Trainingsdatensatz: 01.07.2013 bis 31.07.2017
train_data = data[(data['Datum      '] >= '2013-07-01') & (data['Datum      '] <= '2017-07-31')]

# Validierungsdatensatz: 01.08.2017 bis 31.07.2018
validation_data = data[(data['Datum      '] >= '2017-08-01') & (data['Datum      '] <= '2018-07-31')]

# Beispiel: Ziel ist "Umsatz"
X_train = train_data.drop(columns=['Datum      ','Umsatz              ','Wettercode '])
y_train = train_data['Datum      ']

X_validation = validation_data.drop(columns=['Datum      ','Umsatz              ','Wettercode '])
y_validation = validation_data['Datum      ']

from sklearn.metrics import mean_squared_error, r2_score

# Baseline-Modell: Durchschnitt als Vorhersage
baseline_prediction = y_train.mean()

# Vorhersagen für den Validierungsdatensatz
y_pred_baseline = [baseline_prediction] * len(y_validation)

# Bewertung des Baseline-Modells
mse = mean_squared_error(y_validation, y_pred_baseline)
r2 = r2_score(y_validation, y_pred_baseline)

print(f"Baseline MSE: {mse:.2f}")
print(f"Baseline R²: {r2:.2f}")

ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " ", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.